Copyright **`(c)`** 2025 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [1]:
from copy import deepcopy
from random import random, randint, choice

from tqdm.auto import tqdm
from icecream import ic

In [2]:
# Thanks to Andrea!

In [3]:
NUM_ITEMS = 50
NUM_CATEGORIES = 10

In [ ]:
ITEMS = [[(c, n) for n in range(NUM_ITEMS)] for c in range(NUM_CATEGORIES)]
INCOMPATIBILITIES = [
    {
        (randint(0, NUM_CATEGORIES - 1), randint(0, NUM_ITEMS - 1)),
        (randint(0, NUM_CATEGORIES - 1), randint(0, NUM_ITEMS - 1)),
    }
    for _ in range(10000)
]


def x(i: set) -> bool:
    if len(i) == 1:
        return False
    i1, i2 = sorted(i)
    return i1[0] == i2[0]


INCOMPATIBILITIES = [i for i in INCOMPATIBILITIES if not x(i)]

In [22]:
def invalid(bag: set) -> bool:
    return any(i <= bag for i in INCOMPATIBILITIES)


def cost(solution: list[set]) -> int:
    return sum(invalid(b) for b in solution)

In [23]:
def tweak(bag: list[set], strength: float = 0.1) -> list[set]:
    new_bag = deepcopy(bag)
    again = True
    while again:
        b1 = choice([i for i in range(NUM_ITEMS) if invalid(bag[i])])
        b2 = randint(0, NUM_ITEMS - 1)
        b1_, b2_ = sorted(new_bag[b1]), sorted(new_bag[b2])
        i = randint(0, NUM_CATEGORIES - 1)
        b1_[i], b2_[i] = b2_[i], b1_[i]
        new_bag[b1], new_bag[b2] = set(b1_), set(b2_)
        again = random() < strength
    return new_bag

In [28]:
current_solution = [{ITEMS[c][i] for c in range(NUM_CATEGORIES)} for i in range(NUM_ITEMS)]
current_cost = cost(current_solution)
ic(current_cost)
# randomize it

MAX_STEPS = 500

# SA
temp = 100
for steps in tqdm(range(MAX_STEPS)):
    if current_cost == 0:
        break

    if steps % 100 == 0:
        temp *= .9

    new_solution = tweak(current_solution, strength=0.4)
    new_cost = cost(new_solution)

    if new_cost < current_cost:
        current_cost = new_cost
        current_solution = new_solution
        ic(steps, current_cost)
    elif new_cost == current_cost:
        current_solution = new_solution
    else:
        # SA
        diff = new_cost - current_cost
        p = 2 ** -(temp * diff)
        if random() < p:
            ic(p, new_cost, current_cost)
            current_solution = new_solution

print(current_cost)

ic| current_cost: 50


  0%|          | 0/500 [00:00<?, ?it/s]

ic| steps: 5, current_cost: 49
ic| steps: 16, current_cost: 48
ic| steps: 23, current_cost: 47
ic| steps: 31, current_cost: 46
ic| steps: 83, current_cost: 45
ic| steps: 98, current_cost: 44
ic| steps: 186, current_cost: 43
ic| steps: 226, current_cost: 42
ic| steps: 284, current_cost: 41
ic| steps: 341, current_cost: 40
ic| steps: 343, current_cost: 39
ic| steps: 426, current_cost: 38
ic| steps: 454, current_cost: 37


37
